In [3]:
import os
import time
import glob
import numpy as np
import pandas as pd
from pytrends.request import TrendReq
import id_mappings
import translations

#Setup
curr_dir = os.getcwd()
pytrend = TrendReq(timeout=(10,60))

In [14]:
#Schema for raw data per country

#    country                                str
#    region_name                            str
#    local_id                               str
#    iso_id                                 str
#    date                              datetime
#    population                             int
#    daily_cases                            int
#    daily_deaths                           int
#    total_cases                            int
#    total_deaths                           int
#    loss_smell                         float64
#    loss_smell_daily_weighting         float64
#    sense_smell                          int64
#    sense_smell_daily_weighting          int64
#    loss_taste                         float64
#    loss_taste_daily_weighting         float64
#    sense_taste                          int64
#    sense_taste_daily_weighting          int64

In [15]:
#Italian COVID data uses regional coding system rather than ISO codes. See id_mappings.
#Trentino-Alto Adige (local_iod: 04) is subdivided into 2 regions (local_ids: 21 and 22)...
#...These subregions need to be mapped to local_id: 04 and grouped
#First date of recording at 224 cases: 2020-02-24

def italy_covid():
    italy_file_path = os.path.join(curr_dir, 'raw_covid_data/italy/italy_covid_raw.csv')
    result = pd.read_csv(italy_file_path, index_col=None, usecols=['data', 'codice_regione',\
                                'totale_casi', 'nuovi_positivi', 'deceduti'], dtype={'codice_regione': str,\
                                'denominazione_regione': str, 'totale_casi': int, 'nuovi_positivi': int, 'deceduti': int},\
                                parse_dates=['data'])
    result = result.rename(columns={'codice_regione': 'local_id', 'totale_casi': 'total_cases',\
                              'nuovi_positivi': 'daily_cases', 'deceduti': 'total_deaths'})
    result['country_id'] = 'IT'
    result['date'] = result['data'].dt.normalize()
    result = result.drop(['data'], axis=1)
    result = result.replace({'local_id': {'21': '04', '22': '04'}})
    result = result.groupby(['local_id', 'date']).sum().reset_index()
    result['daily_deaths'] = result.apply(lambda x: x['total_deaths'] - result['total_deaths'].loc[(result['date'] == x['date'] + pd.DateOffset(-1)) &\
                                                                            (result['local_id'] == x['local_id'])].max(), axis=1)
    
    italy_pop_file_path = os.path.join(curr_dir, 'raw_covid_data/italy/italy_population.csv')
    pop_raw = pd.read_csv(italy_pop_file_path, index_col=None, usecols=['local_id', 'total'], dtype={'local_id': str, 'total': int})
    pop_raw = pop_raw.rename(columns={'total': 'population'})
    
    result = result.merge(pop_raw, how='left', left_on='local_id', right_on='local_id')
    return result

In [16]:
#US total cases reach 100 on 2020-03-04

def us_covid():
    #Cases unused cols...
    #UID,iso2,iso3,code3,FIPS,Admin2,Country_Region,Lat,Long_,Combined_Key
    us_cases_file_path = os.path.join(curr_dir, 'raw_covid_data/us/us_covid_raw_cases.csv')
    us_cases_raw = pd.read_csv(us_cases_file_path, index_col=None)
    us_cases_raw_filtered = us_cases_raw.drop(['UID','iso2','iso3','code3','FIPS','Admin2',\
                                               'Country_Region','Lat','Long_','Combined_Key'], axis=1)
    us_cases_raw_grouped = us_cases_raw_filtered.groupby(['Province_State']).sum().reset_index()
    us_cases_df = pd.DataFrame(columns=['region_name', 'date', 'total_cases'])
    for state in id_mappings.us:
        state_cases = us_cases_raw_grouped.loc[us_cases_raw_grouped['Province_State'] == state]
        state_transposed = state_cases.drop(['Province_State'], axis=1).T.reset_index()
        state_renamed = state_transposed.rename(columns={state_transposed.columns[0]: 'date_raw', state_transposed.columns[1]: 'total_cases'})
        state_renamed['region_name'] = state
        state_renamed['date'] = pd.to_datetime(state_renamed['date_raw'], format='%m/%d/%y')
        state_final = state_renamed.drop(['date_raw'], axis=1)
        us_cases_df = us_cases_df.append(state_final)
    us_cases_df = us_cases_df.set_index(['date', 'region_name'])


    #Deaths unused cols...
    #UID,iso2,iso3,code3,FIPS,Admin2,Country_Region,Lat,Long_,Combined_Key,Population
    us_deaths_file_path = os.path.join(curr_dir, 'raw_covid_data/us/us_covid_raw_deaths.csv')
    us_deaths_raw = pd.read_csv(us_deaths_file_path, index_col=None)
    us_deaths_raw_filtered = us_deaths_raw.drop(['UID','iso2','iso3','code3','FIPS','Admin2','Country_Region',\
                                                 'Lat','Long_','Combined_Key','Population'], axis=1)
    us_deaths_raw_grouped = us_deaths_raw_filtered.groupby(['Province_State']).sum().reset_index()
    us_deaths_df = pd.DataFrame(columns=['region_name', 'date', 'total_deaths'])
    for state in id_mappings.us:
        state_deaths = us_deaths_raw_grouped.loc[us_deaths_raw_grouped['Province_State'] == state]
        state_transposed = state_deaths.drop(['Province_State'], axis=1).T.reset_index()
        state_renamed = state_transposed.rename(columns={state_transposed.columns[0]: 'date_raw', state_transposed.columns[1]: 'total_deaths'})
        state_renamed['region_name'] = state
        state_renamed['date'] = pd.to_datetime(state_renamed['date_raw'], format='%m/%d/%y')
        state_final = state_renamed.drop(['date_raw'], axis=1)
        us_deaths_df = us_deaths_df.append(state_final)
    us_deaths_df = us_deaths_df.set_index(['date', 'region_name'])

    result = us_cases_df.merge(us_deaths_df, how='left', left_index=True, right_index=True).reset_index()
    
    result['daily_cases'] = result.apply(lambda x: x['total_cases'] - result['total_cases'].loc[(result['date'] == x['date'] + pd.DateOffset(-1)) &\
                                                                        (result['region_name'] == x['region_name'])].max(), axis=1)
    
    result['daily_deaths'] = result.apply(lambda x: x['total_deaths'] - result['total_deaths'].loc[(result['date'] == x['date'] + pd.DateOffset(-1)) &\
                                                                        (result['region_name'] == x['region_name'])].max(), axis=1)
    
    #Add population data
    us_population_data = us_deaths_raw[['Province_State', 'Population']].groupby(['Province_State']).sum().reset_index()
    us_population_data = us_population_data.rename(columns={'Province_State': 'region_name', 'Population': 'population'})
    result = result.merge(us_population_data, how='left', left_on='region_name', right_on='region_name')
    
    
    return result

In [17]:
#Spanish data has iso-8859-1 encoding
#Date is DD/MM for using dayfirst=True when reading csv
#Spainsh raw data file contains a series of notes at the end within the csv
#Data provided contains total running deaths and cases

def spain_covid():
    spain_file_path = os.path.join(curr_dir, 'raw_covid_data/spain/spain_covid_raw.csv')
    result = pd.read_csv(spain_file_path, encoding='iso-8859-1', index_col=None, usecols=['CCAA','FECHA','PCR+','Fallecidos'],\
                         dtype={'CCAA': str, 'PCR+': float, 'Fallecidos': float}, parse_dates=['FECHA'], dayfirst=True)
    result = result.rename(columns={'CCAA': 'local_id', 'FECHA': 'date', 'PCR+': 'total_cases', 'Fallecidos': 'total_deaths'})
    #Remove comments from csv
    result = result.loc[result['date'].notna() == True]
    result['daily_cases'] = result.apply(lambda x: x['total_cases'] - result['total_cases'].loc[(result['date'] == x['date'] + pd.DateOffset(-1)) &\
                                                                        (result['local_id'] == x['local_id'])].max(), axis=1)
    
    result['daily_deaths'] = result.apply(lambda x: x['total_deaths'] - result['total_deaths'].loc[(result['date'] == x['date'] + pd.DateOffset(-1)) &\
                                                                        (result['local_id'] == x['local_id'])].max(), axis=1)
    
    #Add population data
    spain_pop_file_path = os.path.join(curr_dir, 'raw_covid_data/spain/spain_population.csv')
    pop_raw = pd.read_csv(spain_pop_file_path, index_col=None, usecols=['local_id', 'population'], dtype={'local_id': str, 'population': int})
    result = result.merge(pop_raw, how='left', left_on='local_id', right_on='local_id')
    return result

In [18]:
#Google trends is using the pre-2016 French regional system
#...COVID sata rebuilt to pre-2016 regions by departement
#Unable to find French daily mortility statistics - only z-scores for execss death by post-2016 region available

def france_covid():
    france_file_path = os.path.join(curr_dir, 'raw_covid_data/france/france_covid_raw.csv')
    result = pd.read_csv(france_file_path, index_col=None, usecols=['dep', 'date_de_passage', 'sursaud_cl_age_corona', 'nbre_pass_corona'], dtype={'dep': str,\
                        'sursaud_cl_age_corona': str, 'nbre_pass_corona': float},\
                        parse_dates=['date_de_passage'])
    sub_regions = list(id_mappings.france_sub_regions.keys())
    result = result.loc[result['dep'].isin(sub_regions)]
    result = result.rename(columns={'date_de_passage': 'date', 'nbre_pass_corona': 'daily_cases'})
    result['iso_id'] = result.apply(lambda x: id_mappings.france_sub_regions[x['dep']], axis=1)
    result = result.loc[result['sursaud_cl_age_corona'] =='0']
    result = result[['iso_id', 'date', 'daily_cases']].groupby(['iso_id', 'date']).sum().reset_index()
    result['total_cases'] = result.apply(lambda x: result['daily_cases'].loc[(result['date'] <= x['date']) &\
                                                            (result['iso_id'] == x['iso_id'])].sum(), axis=1)
    result['daily_deaths'] = np.nan
    result['total_deaths'] = np.nan
    
    france_pop_file_path = os.path.join(curr_dir, 'raw_covid_data/france/france_population.csv')
    pop_raw = pd.read_csv(france_pop_file_path, index_col=None, usecols=['departement', 'total'], dtype={'departement': str, 'total': int})
    pop_raw = pop_raw.loc[pop_raw['departement'].isin(sub_regions)]
    pop_raw['iso_id'] = pop_raw.apply(lambda x:id_mappings.france_sub_regions[x['departement']], axis=1)
    pop_grouped = pop_raw[['iso_id', 'total']].groupby(['iso_id']).sum().reset_index()
    pop_grouped = pop_grouped.rename(columns={'total': 'population'})
    
    result = result.merge(pop_grouped, left_on='iso_id', right_on='iso_id')
    
    return result

In [19]:
#Raw data as large excel file with national, state and municipal data
#National data has state id and municipal id fields missing
#State level has municipal id field missing
#Population data from 2019 census is contained within the COVID-19 excel

def brazil_covid():
    brazil_file_path = os.path.join(curr_dir, 'raw_covid_data/brazil/brazil_covid_raw.xlsx')
    raw_df = pd.read_excel(brazil_file_path, index_col=None, header=0, usecols=['estado', 'codmun', 'data',\
                            'populacaoTCU2019', 'casosAcumulado', 'obitosAcumulado'], dtype={'estado': str,\
                            'codmun': str, 'populacaoTCU2019': float, 'casosAcumulado': int, 'obitosAcumulado': int}, parse_dates=['data'])
    raw_df = raw_df.loc[(raw_df['estado'].isna() == False) & (raw_df['codmun'].isna() == True)]
    raw_df = raw_df.drop(['codmun'], axis=1)
    raw_df = raw_df.rename(columns={'estado': 'local_id', 'data': 'date', 'populacaoTCU2019': 'population',\
                                    'casosAcumulado': 'total_cases', 'obitosAcumulado': 'total_deaths'})
    
    result = pd.DataFrame(columns=['local_id', 'date', 'population', 'total_cases', 'total_deaths'])
    set_dates = pd.DataFrame(pd.date_range(start='2020-02-26', end='2020-05-17', name='date'), columns=['date'])
    set_dates = set_dates.reset_index(drop=True)
    regions = raw_df['local_id'].unique()
    for r in regions:
        region_data = raw_df.loc[raw_df['local_id'] == r]
        fill_dates = set_dates.merge(region_data, how='left', left_on='date', right_on='date')
        fill_dates['local_id'] = r
        fill_dates['population'] = region_data['population'].max()
        fill_dates = fill_dates.fillna(0)
        result = result.append(fill_dates)
        
    result['daily_cases'] = result.apply(lambda x: x['total_cases'] - result['total_cases'].loc[(result['date'] == x['date'] + pd.DateOffset(-1)) &\
                                                                        (result['local_id'] == x['local_id'])].max(), axis=1)
    
    result['daily_deaths'] = result.apply(lambda x: x['total_deaths'] - result['total_deaths'].loc[(result['date'] == x['date'] + pd.DateOffset(-1)) &\
                                                                        (result['local_id'] == x['local_id'])].max(), axis=1)
    
    return result

In [20]:
#Format of start/end date 'yyyy-mm-dd'
def get_search_data(phrase_list, regions, country_code, start_day, end_day):
    date_range = start_day + ' ' + end_day
    counter = 0
    results = pd.DataFrame()
    set_dates = pd.DataFrame(pd.date_range(start=start_day, end=end_day, name='date').date, columns=['date'])
    for r in regions:
        df_builder = set_dates
        df_builder['country'] = country_code
        df_builder['iso_id'] = regions[r]['iso_id']
        df_builder['region_name'] = r
        try:
            df_builder['local_id'] = regions[r]['local_id']
        except:
            df_builder['local_id'] = np.nan 
        df_builder = df_builder.set_index(['date', 'iso_id'])
        results = results.append(df_builder)
    
    for p in phrase_list:
        search_term = phrase_list[p]
        phrase_results = pd.DataFrame()
        weightings = pd.DataFrame()
        for r in regions:
            geo_id = regions[r]['iso_id']
            pytrend.build_payload([search_term], timeframe=date_range, geo=geo_id)
            trends_result = pytrend.interest_over_time()
            trends_result = trends_result.rename(columns={search_term: p})
            try:
                trends_result = trends_result.drop(['isPartial'], axis=1)
            except:
                pass
            trends_result['iso_id'] = geo_id
            trends_result = trends_result.set_index([trends_result.index, 'iso_id'])
            phrase_results = phrase_results.append(trends_result)
            time.sleep(4)
            counter += 1
            print(counter, ' / ', (len(phrase_list) * (len(regions) + set_dates['date'].count())))
        for day in set_dates['date']:
            day_end = (day + pd.DateOffset(1))
            start_string = day.strftime('%Y-%m-%d')
            end_string = day_end.strftime('%Y-%m-%d')
            day_range = start_string + ' ' + end_string
            pytrend.build_payload([search_term], timeframe=day_range, geo=country_code)
            daily_weighting = pytrend.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=True)
            daily_weighting['date'] = day
            phrase_col_name = p + '_daily_weighting'
            daily_weighting = daily_weighting.rename(columns={search_term: phrase_col_name})
            daily_weighting = daily_weighting.rename(columns={'geoCode': 'iso_id'})
            daily_weighting = daily_weighting.set_index(['iso_id', 'date'])
            weightings = weightings.append(daily_weighting)
            time.sleep(2)
            counter += 1
            print(counter, ' / ', (len(phrase_list) * (len(regions) + set_dates['date'].count())))
        phrase_results = phrase_results.merge(weightings, how='left', left_index=True, right_index=True)
        results = results.merge(phrase_results, how='left', left_index=True, right_index=True)
    return results.reset_index()

In [10]:
#Italy
def italy_raw():
    italy_covid_raw = italy_covid()
    italy_search_interest = get_search_data(translations.italy, id_mappings.italy, 'IT', '2020-02-24', '2020-05-17')
    result = italy_search_interest.merge(italy_covid_raw, how='left', left_on=['local_id', 'date'], right_on=['local_id', 'date'])
    result.to_csv(os.path.join(os.path.dirname(curr_dir), 'analysis/italy_full.csv'), index=False)
    return result

#US
def us_raw():
    us_covid_raw = us_covid()
    us_search_interest = get_search_data(translations.us, id_mappings.us, 'US', '2020-02-21', '2020-05-17')
    result = us_search_interest.merge(us_covid_raw, how='left', left_on=['date', 'region_name'], right_on=['date', 'region_name'])
    result.to_csv(os.path.join(os.path.dirname(curr_dir), 'analysis/us_full.csv'), index=False)
    return result

#Spain
def spain_raw():
    spain_covid_raw = spain_covid()
    spain_search_interest = get_search_data(translations.spain, id_mappings.spain, 'ES', '2020-02-20', '2020-05-17')
    result = spain_search_interest.merge(spain_covid_raw, how='left', left_on=['date', 'local_id'], right_on=['date', 'local_id'])
    result.to_csv(os.path.join(os.path.dirname(curr_dir), 'analysis/spain_full.csv'), index=False)
    return result

#France
def france_raw():
    france_covid_raw = france_covid()
    france_search_interest = get_search_data(translations.france, id_mappings.france, 'FR', '2020-02-24', '2020-05-17')
    result = france_search_interest.merge(france_covid_raw, how='left', left_on=['date', 'iso_id'], right_on=['date', 'iso_id'])
    result.to_csv(os.path.join(os.path.dirname(curr_dir), 'analysis/france_full.csv'), index=False)
    return result

#Brazil
def brazil_raw():
    brazil_covid_raw = brazil_covid()
    brazil_search_interest = get_search_data(translations.brazil, id_mappings.brazil, 'BR', '2020-02-26', '2020-05-17')
    result = brazil_search_interest.merge(brazil_covid_raw, how='left', left_on=['date', 'local_id'], right_on=['date', 'local_id'])
    result.to_csv(os.path.join(os.path.dirname(curr_dir), 'analysis/brazil_full.csv'), index=False)
    return result

In [16]:
#Maximum number of API requests in 24hrs for Google Trends is 1600
#Check request numbers before attempting to use all raw data functions on the same day

#Uncomment a line below to fetch the raw data and save to the 'analysis' folder, for the respective country
#italy_raw()
#us_raw()
#spain_raw()
#france_raw()
#brazil_raw()